Since the data size is huge for SPLiT-seq, we performed analysis with a R script with qsub.

```
qsub -l s_vmem=500G,mem_req=500G,lmem -cwd -e GE_log/ -o GE_log/ -N analyze_SPLiT_roundtrip script/run_analyze_SPLiT_cnt250.sh
```

where the shell script execute the following R script:

```
library(Seurat)
library(Matrix)
library(tidyverse)

args <- commandArgs(trailingOnly = TRUE)

source("../../util/sc_preprocess.R")

dge_base   <- readRDS("vFeb2021_SPLiT_original_rawdge.rds")
dge_todrop <- readRDS("SPLiT_roundtrip.split.rds")

print("refreshing DGE")
dge_todrop        <- refresh_dge(dge_todrop,
                                 direct = T,
                                 header = F,
                                 src = 2,
                                 dest = 3,
                                 path = "barcode_correspondence_data/FigS3/SPLiT/roundtrip/merged_combine.tsv")

cell.inter <- intersect(colnames(dge_base),colnames(dge_todrop))
dge_base <- dge_base[,cell.inter]
dge_todrop <- dge_todrop[,cell.inter]

dge_target_list <- list(dge_todrop)

print("determining cell set")
use_cells <- get_cells_used(dge_base,dge_target_list,percent_mito = 5,min_features=250)

dge_base   <- dge_base[,use_cells]
dge_todrop <- dge_todrop[,use_cells]

print(paste("dge_base",ncol(dge_base)))
print(paste("dge_todrop",ncol(dge_todrop)))

print("\nstart seurat")
orig.intersect.seu   <- CreateSeuratObject(dge_base)
todrop.intersect.seu <- CreateSeuratObject(dge_todrop)

high_var_genes <- getVarGenes_untilPCA(orig.intersect.seu,num_varFeatures = 5000)

print("running PCA")
orig.intersect.seu   <- getVarGenes_untilPCA(orig.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)
todrop.intersect.seu <- getVarGenes_untilPCA(todrop.intersect.seu,var_genes=high_var_genes,num_varFeatures = 5000)

print("running UMAP")
orig.intersect.seu   <- seurat_processing_UMAP(orig.intersect.seu,dimlimit = 20,res = 0.6)
todrop.intersect.seu <- seurat_processing_UMAP(todrop.intersect.seu,dimlimit = 20,res = 0.6)

store_dir <- "/home/ha5656/work/Interstellar_test/REVISION_RND1/round_trip/intermediate_data/"

print("saving RDS")
saveRDS(orig.intersect.seu,paste0(store_dir,"Aug2022_processed_SPLiT_original_Seurat.intersectionWithRndTrip.rds"))
saveRDS(todrop.intersect.seu,paste0(store_dir,"Aug2022_processed_SPLiT_roundtrip_Seurat.rds"))

#Fig drawing
draw_umap(seu_base = orig.intersect.seu,
          seu_to10x = orig.intersect.seu,
          seu_todrop = todrop.intersect.seu,
          col_seed = 6,
          w=6,
          h=6,
          outdir = "~/work/Interstellar_test/REVISION_RND1/round_trip/downstream_analysis/SPLiT/",
          outname = "Aug2022_SPLiT_roundtrip")

print("CCdist...")
v.vln.todrop <- get_dist_pcaspace_scatter(orig.intersect.seu,
                                          todrop.intersect.seu,
                                          outdir = "~/work/Interstellar_test/REVISION_RND1/round_trip/downstream_analysis/SPLiT/",
                                          outname = "Aug2022_SPLiT_to_roundtrip")

```